# Grocery Store Example

Lucerne University of Applied Sciences and Arts - School of Information Technology

A kid goes into a grocery store and buys four items.
The cashier charges `$7.11`, the kid pays and is about to leave when the cashier calls the kid back and says:
"Hold on, I multiplied the four items instead of adding them. I'll try again.
 Hah, with adding them the price still comes to `$7.11.`"
What were the prices of the four items?

@author: Marc Pouly

Imports

In [14]:
from ortools.sat.python import cp_model

Create constraint model

In [15]:
model = cp_model.CpModel()

One variable for each product

In [16]:
p1 = model.NewIntVar(0, 711, 'p1')
p2 = model.NewIntVar(0, 711, 'p2')
p3 = model.NewIntVar(0, 711, 'p3')
p4 = model.NewIntVar(0, 711, 'p4')

Prices add up to 711

In [17]:
_ = model.Add(p1 + p2 + p3 + p4 == 711)

In [18]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE:
    print(f"Product 1: ${solver.Value(p1)/100:.2f}")
    print(f"Product 2: ${solver.Value(p2)/100:.2f}")
    print(f"Product 3: ${solver.Value(p3)/100:.2f}")
    print(f"Product 4: ${solver.Value(p4)/100:.2f}")

Product 1: $0.00
Product 2: $0.00
Product 3: $0.00
Product 4: $7.11


In [19]:
# This line would do the job ...
# model.AddMultiplicationEquality(711 * 100 * 100 * 100, [p1, p2, p3, p4])
# We get a kernel crash with or-tools version: 7.5.7466
# Check failed: vars.size() == 2 (4 vs. 2) General int_prod not supported yet.
# Google implemented this only for arrays of size 2.
# Here is a quick and dirty work around:

r1 = model.NewIntVar(0, 711 * 711, 'r1')
model.AddMultiplicationEquality(r1, [p1, p2])

r2 = model.NewIntVar(0, 711 * 711, 'r2')
model.AddMultiplicationEquality(r2, [p3, p4])

model.AddMultiplicationEquality(711 * 100 * 100 * 100, [r1, r2])

#### Search a Single Solution with Customized Printing

In [20]:
status = solver.Solve(model)

if status == cp_model.FEASIBLE:
    print(f"Product 1: ${solver.Value(p1)/100:.2f}")
    print(f"Product 2: ${solver.Value(p2)/100:.2f}")
    print(f"Product 3: ${solver.Value(p3)/100:.2f}")
    print(f"Product 4: ${solver.Value(p4)/100:.2f}")

Product 1: $1.25
Product 2: $1.20
Product 3: $3.16
Product 4: $1.50


#### Search a Single Solution with Default Printing

In [21]:
callback = cp_model.VarArraySolutionPrinter([p1, p2, p3, p4])
status   = solver.SolveWithSolutionCallback(model, callback)

Solution 0, time = 0.69 s
  p1 = 125   p2 = 120   p3 = 316   p4 = 150 


#### Search all Solutions with Default Printing

In [22]:
callback = cp_model.VarArraySolutionPrinter([p1, p2, p3, p4])
status   = solver.SearchForAllSolutions(model, callback)

Solution 0, time = 0.83 s
  p1 = 125   p2 = 120   p3 = 316   p4 = 150 
Solution 1, time = 0.85 s
  p1 = 120   p2 = 125   p3 = 316   p4 = 150 
Solution 2, time = 0.85 s
  p1 = 120   p2 = 125   p3 = 150   p4 = 316 
Solution 3, time = 0.85 s
  p1 = 150   p2 = 125   p3 = 316   p4 = 120 
Solution 4, time = 0.86 s
  p1 = 125   p2 = 120   p3 = 150   p4 = 316 
Solution 5, time = 0.86 s
  p1 = 150   p2 = 120   p3 = 316   p4 = 125 
Solution 6, time = 0.87 s
  p1 = 150   p2 = 120   p3 = 125   p4 = 316 
Solution 7, time = 0.87 s
  p1 = 316   p2 = 120   p3 = 150   p4 = 125 
Solution 8, time = 0.87 s
  p1 = 316   p2 = 120   p3 = 125   p4 = 150 
Solution 9, time = 0.88 s
  p1 = 150   p2 = 125   p3 = 120   p4 = 316 
Solution 10, time = 0.95 s
  p1 = 125   p2 = 150   p3 = 316   p4 = 120 
Solution 11, time = 0.97 s
  p1 = 316   p2 = 125   p3 = 150   p4 = 120 
Solution 12, time = 0.99 s
  p1 = 316   p2 = 125   p3 = 120   p4 = 150 
Solution 13, time = 1.02 s
  p1 = 120   p2 = 150   p3 = 125   p4 = 316 
So

#### Search all Solutions with Customized Printing

In [23]:
class MySolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        
    def on_solution_callback(self):
        print(f"Product 1: ${self.Value(self.__variables[0])/100:.2f}")
        print(f"Product 2: ${self.Value(self.__variables[1])/100:.2f}")
        print(f"Product 3: ${self.Value(self.__variables[2])/100:.2f}")
        print(f"Product 4: ${self.Value(self.__variables[3])/100:.2f}\n")
        

callback = MySolutionPrinter([p1, p2, p3, p4])
status   = solver.SearchForAllSolutions(model, callback)

Product 1: $1.25
Product 2: $1.20
Product 3: $3.16
Product 4: $1.50

Product 1: $1.20
Product 2: $1.25
Product 3: $3.16
Product 4: $1.50

Product 1: $1.20
Product 2: $1.25
Product 3: $1.50
Product 4: $3.16

Product 1: $1.50
Product 2: $1.25
Product 3: $3.16
Product 4: $1.20

Product 1: $1.25
Product 2: $1.20
Product 3: $1.50
Product 4: $3.16

Product 1: $1.50
Product 2: $1.20
Product 3: $3.16
Product 4: $1.25

Product 1: $1.50
Product 2: $1.20
Product 3: $1.25
Product 4: $3.16

Product 1: $3.16
Product 2: $1.20
Product 3: $1.50
Product 4: $1.25

Product 1: $3.16
Product 2: $1.20
Product 3: $1.25
Product 4: $1.50

Product 1: $1.50
Product 2: $1.25
Product 3: $1.20
Product 4: $3.16

Product 1: $1.25
Product 2: $1.50
Product 3: $3.16
Product 4: $1.20

Product 1: $3.16
Product 2: $1.25
Product 3: $1.50
Product 4: $1.20

Product 1: $3.16
Product 2: $1.25
Product 3: $1.20
Product 4: $1.50

Product 1: $1.20
Product 2: $1.50
Product 3: $1.25
Product 4: $3.16

Product 1: $1.20
Product 2: $1.50


#### Symmetry Breaking Constraint

In [24]:
model.Add(p1 <= p2)
model.Add(p2 <= p3)
model.Add(p3 <= p4)

status = solver.SearchForAllSolutions(model, callback)

Product 1: $1.20
Product 2: $1.25
Product 3: $1.50
Product 4: $3.16

